# Предсказание стоимости жилья

В проекте нам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучим модель и сделаем предсказания на тестовой выборке. Для оценки качества модели будем использовать метрики RMSE, MAE и R2.

# Импорт необходимых библиотек и инициализация локальной Spark-сессии

In [1]:
import pandas as pd 
import numpy as np
import pyspark

from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
import seaborn as sns
from pyspark.ml.feature import OneHotEncoder 
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

RANDOM_SEED = 2024

In [2]:
spark = SparkSession.builder \
                    .master("local") \
                    .appName("Price house - Logistic regression") \
                    .getOrCreate()

# Подготовка данных

**В колонках датасета содержатся следующие данные:**

- `longitude` — широта;
- `latitude` — долгота;
- `housing_median_age` — медианный возраст жителей жилого массива;
- `total_rooms` — общее количество комнат в домах жилого массива;
- `total_bedrooms` — общее количество спален в домах жилого массива;
- `population` — количество человек, которые проживают в жилом массиве;
- `households` — количество домовладений в жилом массиве;
- `median_income` — медианный доход жителей жилого массива;
- `median_house_value` — медианная стоимость дома в жилом массиве;
- `ocean_proximity` — близость к океану.

**Прочитаем содержимое файла:**

In [3]:
data_housing = spark.read.option('header', 'true').csv('/datasets/housing.csv', inferSchema = True) 
data_housing.printSchema() 

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



In [4]:
data_housing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

**Выведем типы данных колонок датасета, используя методы pySpark:**

In [5]:
print(pd.DataFrame(data_housing.dtypes, columns=['column', 'type']))

               column    type
0           longitude  double
1            latitude  double
2  housing_median_age  double
3         total_rooms  double
4      total_bedrooms  double
5          population  double
6          households  double
7       median_income  double
8  median_house_value  double
9     ocean_proximity  string


***В наборе данных представлены исключительно числовые переменные, исключение составляет одна категориальная переменная - `ocean_proximity`.***

**Выведем с помощью метода describe() базовые описательные статистики данных в виде таблицы в pandas:**

In [6]:
data_housing.describe().toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20433,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,537.8705525375618,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,421.38507007403115,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


***Ого, в датасете примерно 21 тысяча строк. Максимальная медианная стоимость дома равна полумиллиону долларов. Только данные получены давно — сейчас дома в Калифорнии стоят в несколько раз дороже.***

**Исследуем данные на наличие пропусков:**

In [7]:
columns = data_housing.columns


for column in columns:
    print(column, data_housing.where(F.isnan(column) | F.col(column).isNull()).count())

longitude 0
latitude 0
housing_median_age 0
total_rooms 0
total_bedrooms 207
population 0
households 0
median_income 0
median_house_value 0
ocean_proximity 0


Пропуски обнаружены только в столбце `total_bedrooms` — общее количество спален в домах жилого массива, пропущенные значения составляют около 1% от общего числа строк, и их удаление не повредит нашему исследованию.

In [8]:
data_housing = data_housing.na.drop(subset=["total_bedrooms"])

Проверим применились ли изменения: 

In [9]:
columns = data_housing.columns


for column in columns:
    print(column, data_housing.where(F.isnan(column) | F.col(column).isNull()).count())

longitude 0
latitude 0
housing_median_age 0
total_rooms 0
total_bedrooms 0
population 0
households 0
median_income 0
median_house_value 0
ocean_proximity 0


Да, все отлично!

**Разделим колонки на два типа: числовые и текстовые, которые представляют категориальные данные:**

In [10]:
categorical_cols = ['ocean_proximity']

numerical_cols  = ["longitude", "latitude", "housing_median_age", "total_rooms", 
                   "total_bedrooms", "population", "households", "median_income"]

target = "median_house_value" 

**Далее приступим к трансформации признаков:**

В первую очередь трансформируем категориальные признаки с помощью трансформера StringIndexer. Он переводит текстовые категории в числовое представление, так как большинство ML-алгоритмов работает с числовыми данными. По умолчанию трансформер на вход принимает названия колонок, которые нужно трансформировать, и список названий новых колонок.

In [11]:
indexer = StringIndexer(inputCols=categorical_cols, 
                        outputCols=[c+'_idx' for c in categorical_cols]) 
data_housing = indexer.fit(data_housing).transform(data_housing)

cols = [c for c in data_housing.columns for i in categorical_cols if (c.startswith(i))]
data_housing.select(cols).show(3) 

+---------------+-------------------+
|ocean_proximity|ocean_proximity_idx|
+---------------+-------------------+
|       NEAR BAY|                3.0|
|       NEAR BAY|                3.0|
|       NEAR BAY|                3.0|
+---------------+-------------------+
only showing top 3 rows



Создадим OHE-кодирование (от англ. One hot encoder) для категорий.
OneHotEncoder работает так же, как StringIndexer, — принимает на вход названия колонок, которые нужно трансформировать, и список названий новых колонок.

In [12]:
encoder = OneHotEncoder(inputCols=[c+'_idx' for c in categorical_cols],
                        outputCols=[c+'_ohe' for c in categorical_cols])
data_housing = encoder.fit(data_housing).transform(data_housing)

cols = [c for c in data_housing.columns for i in categorical_cols if (c.startswith(i))]
data_housing.select(cols).show(3) 

+---------------+-------------------+-------------------+
|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|
+---------------+-------------------+-------------------+
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
+---------------+-------------------+-------------------+
only showing top 3 rows



In [13]:
#data_housing.toPandas().head()

In [14]:
data_housing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|        

Финальный шаг преобразований — это объединение признаков в один вектор, с которым ML-алгоритм умеет работать.

In [15]:
categorical_assembler = \
        VectorAssembler(inputCols=[c+'_ohe' for c in categorical_cols],
                                        outputCol="categorical_features")
data_housing = categorical_assembler.transform(data_housing) 

Для числовых признаков тоже нужна трансформация — шкалирование значений — чтобы сильные выбросы не смещали предсказания модели.

In [16]:
numerical_assembler = VectorAssembler(inputCols=numerical_cols,outputCol="numerical_features")
data_housing = numerical_assembler.transform(data_housing)

In [17]:
standardScaler = StandardScaler(inputCol='numerical_features',outputCol="numerical_features_scaled")
data_housing = standardScaler.fit(data_housing).transform(data_housing)

После всех трансформаций получается такая таблица:

In [18]:
print(data_housing.columns)

['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity', 'ocean_proximity_idx', 'ocean_proximity_ohe', 'categorical_features', 'numerical_features', 'numerical_features_scaled']


In [19]:
#data_housing.toPandas().head()


<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Тоже самое
</div>

In [20]:
data_housing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+--------------------+--------------------+-------------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|categorical_features|  numerical_features|numerical_features_scaled|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+--------------------+--------------------+-------------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|                3.0|      (4,[3],[1.0])|       (4,[3],[1.0])|[-122.23,37.88,41...|     [-61.005863841998...|
|  -122.22|   37

Финальный шаг — собрать трансформированные категорийные и числовые признаки с помощью VectorAssembler.

In [21]:
all_features = ['categorical_features','numerical_features_scaled']

final_assembler = VectorAssembler(inputCols=all_features, 
                                  outputCol="features") 

data_housing = final_assembler.transform(data_housing)

data_housing.select(all_features).show(5) 

+--------------------+-------------------------+
|categorical_features|numerical_features_scaled|
+--------------------+-------------------------+
|       (4,[3],[1.0])|     [-61.005863841998...|
|       (4,[3],[1.0])|     [-61.000872770752...|
|       (4,[3],[1.0])|     [-61.010854913244...|
|       (4,[3],[1.0])|     [-61.015845984490...|
|       (4,[3],[1.0])|     [-61.015845984490...|
+--------------------+-------------------------+
only showing top 5 rows



# Обучение моделей

Разделяем наш датасет на две части — выборку для обучения и выборку для тестирования качества модели.

Разделение на выборки в Spark можно сделать с помощью метода randomSplit(). 

In [22]:
train_data, test_data = data_housing.randomSplit([.8,.2], seed=RANDOM_SEED)
print(train_data.count(), test_data.count()) 

16394 4039


Построим две модели линейной регрессии на разных наборах данных:
- используя все данные из файла;
- используя только числовые переменные, исключив категориальные.

Для построения модели будем использовать оценщик LinearRegression из библиотеки MLlib.

**Модель с использованием всех файлов:**

In [23]:
lr_first = LinearRegression(labelCol=target, featuresCol='features')

model_first = lr_first.fit(train_data) 

24/02/14 14:30:26 WARN Instrumentation: [e8a7b02b] regParam is zero, which might cause numerical instability and overfitting.
24/02/14 14:30:26 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/02/14 14:30:26 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
24/02/14 14:30:27 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
24/02/14 14:30:27 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [24]:
predictions_first = model_first.transform(test_data)

predictedLabes = "prediction"

**Модель с использованием только числовых переменных, исключив категориальные:**

In [25]:
lr_second = LinearRegression(labelCol=target, featuresCol='numerical_features_scaled')

model_second = lr_second.fit(train_data) 

24/02/14 14:30:29 WARN Instrumentation: [cd91db76] regParam is zero, which might cause numerical instability and overfitting.


In [26]:
predictions_second = model_second.transform(test_data)

# Анализ результатов

Сравним результаты работы линейной регрессии на двух наборах данных по метрикам RMSE, MAE и R2. 

Сделаем выводы.

Для задачи регрессии используется RegressionEvaluator:


https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.evaluation.RegressionEvaluator.html

In [27]:
evaluator = RegressionEvaluator(predictionCol=predictedLabes, labelCol = target)

**RMSE — это корень среднего квадрата ошибок, чем ниже RMSE, тем лучше модель может «соответствовать» набору данных:**

***Оценка для первой модели:***

In [28]:
evaluator.evaluate(predictions_first, {evaluator.metricName: "rmse"})

69530.75171864238

***Оценка для второй модели:***

In [29]:
evaluator.evaluate(predictions_second, {evaluator.metricName: "rmse"})

70245.12406852088

**Для первой модели метрика RMSE лучше.**

Средняя абсолютная ошибка (Mean Absolute Error) – это степень несоответствия между фактическими и прогнозируемыми значениями.

Абсолютная ошибка представляет собой разность между спрогнозированным и фактическим значениями. 

***MAE*** — это среднее от таких ошибок, что помогает понять эффективность модели.

Чем ближе MAE к нулю, тем точнее модель. 

***Оценка для первой модели:***

In [30]:
evaluator.evaluate(predictions_first, {evaluator.metricName: "mae"})

50426.79587415038

***Оценка для второй модели:***

In [31]:
evaluator.evaluate(predictions_second, {evaluator.metricName: "mae"})

51483.290718840384

**По метрике MAE первая модель точнее.**

Коэффициент детерминации, или **метрика R2**, вычисляет долю среднеквадратичной ошибки модели от MSE среднего, а затем вычитает эту величину из единицы. Увеличение метрики означает прирост
качества модели.

***Оценка для первой модели:***

In [32]:
evaluator.evaluate(predictions_first, {evaluator.metricName: "r2"})

0.6376158143184116

***Оценка для второй модели:***

In [33]:
evaluator.evaluate(predictions_second, {evaluator.metricName: "r2"})

0.6301311513972139

**Качество первой модели по результам метрики R2 лучше.**

# Вывод

**В ходе выполнения проекта:**

- Инициализировали локальную Spark-сессию.
	
- Прочитали содержимое файла housing.csv.
		
- Вывели типы данных колонок датасета, используя методы pySpark и выяснили что: 
    - В наборе данных представлены исключительно числовые переменные, исключение составила одна категориальная переменная - ocean_proximity.
		
- Выполнили предобработку данных:
  - Исследовали данные на наличие пропусков и выяснили что: 
	- Пропуски обнаружены только в столбце total_bedrooms — общее количество спален в домах жилого массива, пропущенные значения составляют около 1% от общего числа строк, и их удаление не повредило нашему исследованию.
  - Преобразовали колонку с категориальными значениями техникой One hot encoding.
  - Также пришли к выводу, что для числовых признаков тоже нужна трансформация — шкалирование значений — чтобы сильные выбросы не смещали предсказания модели.
		
- Построили две модели линейной регрессии на разных наборах данных:
	- используя все данные из файла;
	- используя только числовые переменные, исключив категориальные;
    - для построения модели использовали оценщик LinearRegression из библиотеки MLlib.
		
- Сравнили результаты работы линейной регрессии на двух наборах данных по метрикам RMSE, MAE и R2. И пришли к выводу, что на всех трех метриках первая модель (с использованием всех данных) показывает более лучшие значения. 